## Example Selection

In [ ]:
import pandas as pd
import json

label_column = 'Class'


train_df = pd.read_csv(f'../../Tasks/germeval/train.csv')
train_df['length'] = train_df['text'].apply(lambda x: len(x))
train_df[label_column] = train_df[label_column].apply(lambda x: 'offensive' if x else 'non-offensive')

context = ""
for j in train_df[label_column].unique():
	print(f'Class: {j}')
	sample = train_df[train_df['length'] < 180]
	sample = sample[sample[label_column] == j].sample(5)
	for index, row in sample.iterrows():
		print(f'  {row["text"]}')
		context += f'TEXT: <text_icl_begin> {row["text"]} <text_icl_end>\nLABEL: {j}\n'

#save the context into a json file
context = f"You are an expert in social psychology.\nDecide whether the **TEXT** is offensive or non-offensive. Reply with the single label on its own line — no extra words.\n\n### EXAMPLES\n{context}### END EXAMPLES"
with open('../icl_promtps/germeval.json', 'w') as f:
	json.dump([context], f, indent=4)

Class: non-offensive
  @ainyrockstar @GiKi56534206 Alles andere wäre ausländerfeindlich ....
  @Karstensailer Zum Ritter geschlagen, weil er viel Geld an Hospitäler gespendet hat - Ablasshandel - vor denen, die am meisten Spenden, sollte man auf der Hut sein -!
  Wenn ein Sprachkünstler wie Wolfgang #Thierse zuschlägt und in @infoammorgen von "Entheimatung" spricht und "Konsense" anmahnt ..... @DLF
  @schalburg1 Hatte die Idee auch schon. Nachzulesen in meinem 2015 geschriebenen Buch "Wutmama"
  Vielleicht sollten wir keine "NATIONAL"-Mannschaften mehr in kämpferische Auseinandersetzungen (WM´s,EM´s u.s.w.) schicken,zu PATRIOTISCH !
Class: offensive
  Alle diejenigen die nicht politisch korrekt sind, sind Verbrecher. So sehen das heutzutage Medien, Politiker, Toleranz Fanatiker sowie bahnhofsklatscher. Verblödetes Deutschland.
  @Heinrich_Krug @ReggyVV Der abgetakelte Legastheniker tut alles für ein paar Dollar mehr, selbst sein Volk verraten, wobei ich nicht sicher bin ob es jemals se

### Pompt GPT-4o model and save results for each fold

In [1]:
import json 

with open('../icl_promtps/germeval.json', 'r') as f:
	data = json.load(f)[0]

In [ ]:
import pandas as pd
import openai, json
import pandas as pd
from tqdm import tqdm
import traceback, pickle

import os, sys, dotenv
dotenv.load_dotenv()

client = openai.OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

def get_response( message ): 
  
    response = client.chat.completions.create(
      model="chatgpt-4o-latest",
      # logprobs = True,
      messages=[
        {"role": "system", "content": "You are an expert in social psychology."},
        {"role": "user", "content": message},
      ],
       max_tokens = 10,

    )
    # print(response.choices[0].message.content)
    return response.choices[0].message.content


def get_inference(message, label_set):
    
	pred = "unknown"
	for i in range(6):
		try:
			z = get_response(message)
			if z.strip().lower() in label_set:
				pred = z
				break
		except:
			print(traceback.format_exc())
			pass

	return pred

df = pd.read_csv(f'../../Tasks/germeval/test.csv')
df = df.dropna(subset=['text'])
df['text+template'] = df['text'].apply(lambda x: f"{data}\n\nTEXT: <text_begin> {x} <text_end>\nLABEL:")

for i in range(4):

	for index, row in tqdm(df.iterrows(), total=len(df)):
		response = get_inference(row['text+template'], label_set=['offensive', 'non-offensive'])
		df.at[index, 'predicted_label'] = response
	
	df.to_csv(f'../outputs/o3-mini/germeval/test_{i}.csv', index=False)
		

In [1]:
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np

f1_scores = []
for i in range(3):
	df = pd.read_csv(f'outputs/germeval/test_{i}.csv')
	df['Class'] = df['Class'].apply(lambda x: 'offensive' if x else 'non-offensive')
	f1_scores += [f1_score(df['Class'], df['predicted_label'], average='macro')]
	
print('Avg', np.mean(f1_scores))
print('Std', np.std(f1_scores))

Avg 0.8538477881795492
Std 0.0016735386435124385
